In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from string import ascii_letters
import matplotlib.pyplot as plt
import datetime as dt
import requests
from lxml import html
import math
import re
import json
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


If you run this, make sure from this shared folder, right click the data files and add short cut to "My Drive". This ensure the path is correct.

In [4]:
airbnb_df = pd.read_csv('/content/drive/MyDrive/airbnb_cleaned.csv')
spotify_df = pd.read_csv('/content/drive/MyDrive/spotify_charts.csv')
ticket_df = pd.read_json('/content/drive/MyDrive/ticketmaster_events.json')

In [5]:
ticket_df

,event_url,event_id,event_category,event_subcategory,title,datetime_utc,datetime_local,name,address,city,state,country,postal_code,formatted_address
0,https://www.ticketmaster.com/absinthe-las-vega...,39005D610179ADFF,ARTS & THEATRE,PERFORMANCE ART,Absinthe,2023/08/02 05:00:00,2023/08/01 22:00:00,The Spiegeltent at Caesars Palace,3570 las vegas blvd south,Las Vegas,NV,US,89109,"3570 Las Vegas Blvd S, Las Vegas, NV 89109, US"
1,https://www.ticketmaster.com/ally-venable-191-...,19005EC2B8982BC8,MUSIC,ROCK,Ally Venable @ 191 Toole,2023/08/02 03:00:00,2023/08/01 20:00:00,191 Toole,191 e toole ave,Tucson,AZ,US,85701,"191 E Toole Ave, Tucson, AZ 85701, US"
2,https://www.ticketmaster.com/des-moines-storyt...,06005D77AE02514A,MISCELLANEOUS,LECTURE/SEMINAR,Des Moines Storytellers Project Season Package,2023/02/15 01:00:00,2023/02/14 19:00:00,Hoyt Sherman Place,1501 woodland avenue,Des Moines,IA,US,50309,"1501 Woodland Ave, Des Moines, IA 50309, US"
3,https://www.ticketmaster.com/fantasy-las-vegas...,2E005D34F9EA55A8,ARTS & THEATRE,PERFORMANCE ART,Fantasy,2023/08/02 05:30:00,2023/08/01 22:30:00,Atrium Showroom at Luxor Hotel and Casino Las ...,3900 las vegas blvd south,Las Vegas,NV,US,89109,"3900 Las Vegas Blvd S, Las Vegas, NV 89119, US"
4,https://www.ticketmaster.com/fred-armisen-come...,0B005ED5E51A297E,ARTS & THEATRE,COMEDY,Fred Armisen - Comedy For Musicians But Everyo...,2023/08/02 03:00:00,2023/08/01 20:00:00,Beverly O'Neill Theater - Long Beach Conv and ...,300 e ocean bl,Long Beach,CA,US,90802,"300 E Ocean Blvd, Long Beach, CA 90802, US"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80048,https://www.ticketmaster.com/jimmie-allen-hend...,17005E34AB3E8572,MUSIC,COUNTRY,Jimmie Allen,2025/05/18 03:00:00,2025/05/17 20:00:00,Sunset Station Outdoor Amphitheater,1301 west sunset road,Henderson,NV,US,89014,"1301 W Sunset Rd, Henderson, NV 89014, US"
80049,https://www.ticketmaster.com/event/Z7r9jZ1Ad-zON,Z7r9jZ1Ad-zON,MUSIC,OTHER,Babytron,2025/05/07 23:00:00,2025/05/07 19:00:00,Webster Theatre,31 webster st,Hartford,CT,US,06114,"31 Webster St, Hartford, CT 06114, US"
80050,https://www.ticketmaster.com/blessd-blessd-cor...,17005E7BCDF965EC,MUSIC,HIP-HOP/RAP,Blessd - Blessd Corner,2025/05/08 03:00:00,2025/05/07 20:00:00,The Cosmopolitan of Las Vegas,3708 las vegas boulevard south,Las Vegas,NV,US,89109,"3708 Las Vegas Blvd S, Las Vegas, NV 89109, US"
80051,https://www.ticketmaster.com/sponsorship-venmo...,20005EFAA94EBF39,MUSIC,POP,Sponsorship - Venmo,2025/06/01 23:30:00,2025/06/01 19:30:00,Presence Arena,555 main st,Atlanta,GA,US,30363,"555 Main St NE, Atlanta, GA 30324, US"


# Summary statistics

# Cleaning airbnb df

In [20]:
airbnb_df

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,city
0,958,"Bright, Modern Garden Unit - 1BR/1BTH",1169,Holly,Western Addition,37.770280,-122.433170,Entire home/apt,202,2,383,2023-02-19,2.31,1,128,59,San Francisco
1,5858,Creative Sanctuary,8904,Philip And Tania,Bernal Heights,37.744740,-122.420890,Entire home/apt,235,30,111,2017-08-06,0.66,1,365,0,San Francisco
2,8142,Friendly Room Apt. Style -UCSF/USF - San Franc...,21994,Aaron,Haight Ashbury,37.765550,-122.452130,Private room,56,32,9,2022-10-27,0.09,13,365,1,San Francisco
3,8339,Historic Alamo Square Victorian,24215,Rosy,Western Addition,37.775640,-122.436420,Entire home/apt,575,9,28,2019-06-28,0.17,2,365,0,San Francisco
4,8739,"Mission Sunshine, with Private Bath",7149,Ivan & Wendy,Mission,37.760300,-122.421970,Private room,110,1,770,2023-02-25,4.65,2,159,34,San Francisco
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232126,849230448624862502,Private Half Moon Bay Oasis By Pillar Point Be...,29395385,Khalid,Unincorporated Areas,37.506164,-122.476188,Entire home/apt,389,2,0,NaN,NaN,1,365,0,San Mateo
232127,850805127216414630,Home near SFO,440957696,Christina,San Bruno,37.623630,-122.426570,Entire home/apt,158,1,0,NaN,NaN,1,364,0,San Mateo
232128,851562104616413652,Modern New Private Studio in SFO,41574587,Vality,South San Francisco,37.655661,-122.413130,Entire home/apt,97,1,0,NaN,NaN,32,359,0,San Mateo
232129,851792795339759410,Lux Bedroom with a Shared Bath near SFO,73133543,Stephannie,South San Francisco,37.644792,-122.454213,Private room,62,2,0,NaN,NaN,3,67,0,San Mateo


In [21]:
# snapshot of airbnb
airbnb_df.head(5)

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,city
0,958,"Bright, Modern Garden Unit - 1BR/1BTH",1169,Holly,Western Addition,37.77028,-122.43317,Entire home/apt,202,2,383,2023-02-19,2.31,1,128,59,San Francisco
1,5858,Creative Sanctuary,8904,Philip And Tania,Bernal Heights,37.74474,-122.42089,Entire home/apt,235,30,111,2017-08-06,0.66,1,365,0,San Francisco
2,8142,Friendly Room Apt. Style -UCSF/USF - San Franc...,21994,Aaron,Haight Ashbury,37.76555,-122.45213,Private room,56,32,9,2022-10-27,0.09,13,365,1,San Francisco
3,8339,Historic Alamo Square Victorian,24215,Rosy,Western Addition,37.77564,-122.43642,Entire home/apt,575,9,28,2019-06-28,0.17,2,365,0,San Francisco
4,8739,"Mission Sunshine, with Private Bath",7149,Ivan & Wendy,Mission,37.76030,-122.42197,Private room,110,1,770,2023-02-25,4.65,2,159,34,San Francisco


In [10]:
airbnb_df[['neighbourhood_group']].nunique()

KeyError: "None of [Index(['neighbourhood_group'], dtype='object')] are in the [columns]"

In [ ]:
airbnb_df[['city']].nunique()

In [ ]:
airbnb_df.info()

In [ ]:
# CLEANING AIRBNB
airbnb_inter = airbnb_df.drop(['neighbourhood_group'], axis=1)
airbnb_inter['city'].isnull().any() # this is false, so there are no null values in the city
airbnb_inter['name'].isnull().any() # this is true - there are 272 null values
airbnb_inter.size

In [ ]:
airbnb_inter = airbnb_inter.dropna(subset=['name'])
airbnb_inter[airbnb_inter['name'].isnull()].size # now it's 0
airbnb_inter['name'].isnull().any()

In [ ]:
# filling in missing values
airbnb_inter[airbnb_inter['host_name'].isnull()]
airbnb_inter[['host_name']] = airbnb_inter[['host_name']].fillna('Unknown Host')

In [11]:
airbnb_inter.isnull().any() # there are null values in last_review, reveiws_per_month
# after inspection, there are only null in last_review, reviews_per_month if reviews is 0

NameError: name 'airbnb_inter' is not defined

In [22]:
airbnb_df.isnull().any()

id                                False
name                              False
host_id                           False
host_name                         False
neighbourhood                     False
latitude                          False
longitude                         False
room_type                         False
price                             False
minimum_nights                    False
number_of_reviews                 False
last_review                        True
reviews_per_month                  True
calculated_host_listings_count    False
availability_365                  False
number_of_reviews_ltm             False
city                              False
dtype: bool

In [23]:
airbnb_df

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,city
0,958,"Bright, Modern Garden Unit - 1BR/1BTH",1169,Holly,Western Addition,37.770280,-122.433170,Entire home/apt,202,2,383,2023-02-19,2.31,1,128,59,San Francisco
1,5858,Creative Sanctuary,8904,Philip And Tania,Bernal Heights,37.744740,-122.420890,Entire home/apt,235,30,111,2017-08-06,0.66,1,365,0,San Francisco
2,8142,Friendly Room Apt. Style -UCSF/USF - San Franc...,21994,Aaron,Haight Ashbury,37.765550,-122.452130,Private room,56,32,9,2022-10-27,0.09,13,365,1,San Francisco
3,8339,Historic Alamo Square Victorian,24215,Rosy,Western Addition,37.775640,-122.436420,Entire home/apt,575,9,28,2019-06-28,0.17,2,365,0,San Francisco
4,8739,"Mission Sunshine, with Private Bath",7149,Ivan & Wendy,Mission,37.760300,-122.421970,Private room,110,1,770,2023-02-25,4.65,2,159,34,San Francisco
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232126,849230448624862502,Private Half Moon Bay Oasis By Pillar Point Be...,29395385,Khalid,Unincorporated Areas,37.506164,-122.476188,Entire home/apt,389,2,0,NaN,NaN,1,365,0,San Mateo
232127,850805127216414630,Home near SFO,440957696,Christina,San Bruno,37.623630,-122.426570,Entire home/apt,158,1,0,NaN,NaN,1,364,0,San Mateo
232128,851562104616413652,Modern New Private Studio in SFO,41574587,Vality,South San Francisco,37.655661,-122.413130,Entire home/apt,97,1,0,NaN,NaN,32,359,0,San Mateo
232129,851792795339759410,Lux Bedroom with a Shared Bath near SFO,73133543,Stephannie,South San Francisco,37.644792,-122.454213,Private room,62,2,0,NaN,NaN,3,67,0,San Mateo


In [24]:
airbnb_df.dropna(subset=['last_review', 'reviews_per_month'], inplace=True)

In [25]:
airbnb_df['last_review_date'] = pd.to_datetime(airbnb_df['last_review'], format='%Y-%m-%d')
airbnb_df.drop(columns=['last_review'])

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,city,last_review_date
0,958,"Bright, Modern Garden Unit - 1BR/1BTH",1169,Holly,Western Addition,37.770280,-122.433170,Entire home/apt,202,2,383,2.31,1,128,59,San Francisco,2023-02-19
1,5858,Creative Sanctuary,8904,Philip And Tania,Bernal Heights,37.744740,-122.420890,Entire home/apt,235,30,111,0.66,1,365,0,San Francisco,2017-08-06
2,8142,Friendly Room Apt. Style -UCSF/USF - San Franc...,21994,Aaron,Haight Ashbury,37.765550,-122.452130,Private room,56,32,9,0.09,13,365,1,San Francisco,2022-10-27
3,8339,Historic Alamo Square Victorian,24215,Rosy,Western Addition,37.775640,-122.436420,Entire home/apt,575,9,28,0.17,2,365,0,San Francisco,2019-06-28
4,8739,"Mission Sunshine, with Private Bath",7149,Ivan & Wendy,Mission,37.760300,-122.421970,Private room,110,1,770,4.65,2,159,34,San Francisco,2023-02-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232091,837369168777380540,Cozy Bedroom for Traveler,417706467,Qiuying,Unincorporated Areas,37.631403,-122.404954,Private room,60,1,2,2.00,4,26,2,San Mateo,2023-03-21
232096,837376796422514183,Lovely Bedroom close to SFO,417706467,Qiuying,Unincorporated Areas,37.629656,-122.404476,Private room,52,1,1,1.00,4,24,1,San Mateo,2023-03-17
232097,837858369497137832,Cozy bedroom in the heart of Silicon Valley,6887379,Jimena,Unincorporated Areas,37.459521,-122.265714,Entire home/apt,59,1,1,1.00,3,252,1,San Mateo,2023-03-11
232098,838031150138816300,"246C - Large Studio w/ Kitchenette, Laundry & ...",250584391,Victor 5,San Mateo,37.528251,-122.297826,Entire home/apt,100,2,1,1.00,18,171,1,San Mateo,2023-03-19


In [26]:
airbnb_df = airbnb_df.drop(columns=['reviews_per_month', 'calculated_host_listings_count', 'number_of_reviews_ltm', 'last_review'])

In [27]:
airbnb_df.rename(columns={'availability_365': 'availability'}, inplace=True)
airbnb_df

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability,city,last_review_date
0,958,"Bright, Modern Garden Unit - 1BR/1BTH",1169,Holly,Western Addition,37.770280,-122.433170,Entire home/apt,202,2,383,128,San Francisco,2023-02-19
1,5858,Creative Sanctuary,8904,Philip And Tania,Bernal Heights,37.744740,-122.420890,Entire home/apt,235,30,111,365,San Francisco,2017-08-06
2,8142,Friendly Room Apt. Style -UCSF/USF - San Franc...,21994,Aaron,Haight Ashbury,37.765550,-122.452130,Private room,56,32,9,365,San Francisco,2022-10-27
3,8339,Historic Alamo Square Victorian,24215,Rosy,Western Addition,37.775640,-122.436420,Entire home/apt,575,9,28,365,San Francisco,2019-06-28
4,8739,"Mission Sunshine, with Private Bath",7149,Ivan & Wendy,Mission,37.760300,-122.421970,Private room,110,1,770,159,San Francisco,2023-02-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232091,837369168777380540,Cozy Bedroom for Traveler,417706467,Qiuying,Unincorporated Areas,37.631403,-122.404954,Private room,60,1,2,26,San Mateo,2023-03-21
232096,837376796422514183,Lovely Bedroom close to SFO,417706467,Qiuying,Unincorporated Areas,37.629656,-122.404476,Private room,52,1,1,24,San Mateo,2023-03-17
232097,837858369497137832,Cozy bedroom in the heart of Silicon Valley,6887379,Jimena,Unincorporated Areas,37.459521,-122.265714,Entire home/apt,59,1,1,252,San Mateo,2023-03-11
232098,838031150138816300,"246C - Large Studio w/ Kitchenette, Laundry & ...",250584391,Victor 5,San Mateo,37.528251,-122.297826,Entire home/apt,100,2,1,171,San Mateo,2023-03-19


In [28]:
airbnb_df.rename(columns={'neighbourhood': 'neighborhood'}, inplace=True)
airbnb_df

,id,name,host_id,host_name,neighborhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability,city,last_review_date
0,958,"Bright, Modern Garden Unit - 1BR/1BTH",1169,Holly,Western Addition,37.770280,-122.433170,Entire home/apt,202,2,383,128,San Francisco,2023-02-19
1,5858,Creative Sanctuary,8904,Philip And Tania,Bernal Heights,37.744740,-122.420890,Entire home/apt,235,30,111,365,San Francisco,2017-08-06
2,8142,Friendly Room Apt. Style -UCSF/USF - San Franc...,21994,Aaron,Haight Ashbury,37.765550,-122.452130,Private room,56,32,9,365,San Francisco,2022-10-27
3,8339,Historic Alamo Square Victorian,24215,Rosy,Western Addition,37.775640,-122.436420,Entire home/apt,575,9,28,365,San Francisco,2019-06-28
4,8739,"Mission Sunshine, with Private Bath",7149,Ivan & Wendy,Mission,37.760300,-122.421970,Private room,110,1,770,159,San Francisco,2023-02-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232091,837369168777380540,Cozy Bedroom for Traveler,417706467,Qiuying,Unincorporated Areas,37.631403,-122.404954,Private room,60,1,2,26,San Mateo,2023-03-21
232096,837376796422514183,Lovely Bedroom close to SFO,417706467,Qiuying,Unincorporated Areas,37.629656,-122.404476,Private room,52,1,1,24,San Mateo,2023-03-17
232097,837858369497137832,Cozy bedroom in the heart of Silicon Valley,6887379,Jimena,Unincorporated Areas,37.459521,-122.265714,Entire home/apt,59,1,1,252,San Mateo,2023-03-11
232098,838031150138816300,"246C - Large Studio w/ Kitchenette, Laundry & ...",250584391,Victor 5,San Mateo,37.528251,-122.297826,Entire home/apt,100,2,1,171,San Mateo,2023-03-19


In [29]:
airbnb_df1 = airbnb_df[['id', 'name', 'host_id', 'neighborhood', 'city', 'room_type', 'number_of_reviews', 'availability', 'minimum_nights', 'price', 'last_review_date']]
airbnb_df2 = airbnb_df[['host_id', 'host_name']]
airbnb_df3 = airbnb_df[['id', 'latitude', 'longitude']]

In [30]:
duplicates = airbnb_df3.duplicated()  # 'keep=False' marks all duplicates as True

# Display duplicate rows
print(airbnb_df3[duplicates])

             id  latitude  longitude
228978  1398981  37.45318 -122.16341


In [31]:
airbnb_df2 = airbnb_df2.drop_duplicates()

In [32]:
airbnb_df3 = airbnb_df3.drop_duplicates()

In [33]:
duplicates = airbnb_df1.duplicated('id', keep=False)  # 'keep=False' marks all duplicates as True

# Display duplicate rows
print(airbnb_df1[duplicates])

             id                     name  host_id          neighborhood  \
111790  1398981  Heart of Silicon Valley    52835             Palo Alto   
228978  1398981  Heart of Silicon Valley    52835  Unincorporated Areas   

               city        room_type  number_of_reviews  availability  \
111790  Santa Clara  Entire home/apt                 26            89   
228978    San Mateo  Entire home/apt                 26            89   

        minimum_nights  price last_review_date  
111790               2    279       2022-05-22  
228978               2    279       2022-05-22  


In [34]:
airbnb_df1 = airbnb_df1.drop(index=228978)

In [37]:
duplicates = airbnb_df2.duplicated('host_id', keep=False)  # 'keep=False' marks all duplicates as True

# Display duplicate rows
print(airbnb_df2[duplicates])

Empty DataFrame
Columns: [host_id, host_name]
Index: []


In [36]:
airbnb_df2 = airbnb_df2.drop(index=167926)
airbnb_df2 = airbnb_df2.drop(index=126913)
airbnb_df2 = airbnb_df2.drop(index=127510)
airbnb_df2 = airbnb_df2.drop(index=229820)
airbnb_df2 = airbnb_df2.drop(index=232009)

In [39]:
duplicates = airbnb_df3.duplicated(['id'], keep=False)  # 'keep=False' marks all duplicates as True

# Display duplicate rows
print(airbnb_df3[duplicates])

Empty DataFrame
Columns: [id, latitude, longitude]
Index: []


In [41]:
airbnb_df3[airbnb_df['id'] == 3052357]

<ipython-input-41-d628071b6e01>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  airbnb_df3[airbnb_df['id'] == 3052357]


,id,latitude,longitude
27862,3052357,34.03281,-118.30496


In [42]:
airbnb_df3

,id,latitude,longitude
0,958,37.770280,-122.433170
1,5858,37.744740,-122.420890
2,8142,37.765550,-122.452130
3,8339,37.775640,-122.436420
4,8739,37.760300,-122.421970
...,...,...,...
232091,837369168777380540,37.631403,-122.404954
232096,837376796422514183,37.629656,-122.404476
232097,837858369497137832,37.459521,-122.265714
232098,838031150138816300,37.528251,-122.297826


In [ ]:
# Check for out of range values
max_int = 2147483647  # Maximum for INT
maxid = airbnb_df1['id'].apply(lambda x: max_int if x > max_int else x)
maxid

# Cleaning Spotify df

In [5]:
# snapshot of spotify
spotify_df.head(250)

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0
...,...,...,...,...,...,...,...,...,...
245,Dancing On My Own,45,2017-01-01,Calum Scott,https://open.spotify.com/track/02WacdrRpm4zlP8...,Australia,top200,MOVE_UP,37738.0
246,Redbone,46,2017-01-01,Childish Gambino,https://open.spotify.com/track/3kxfsdsCpFgN412...,Australia,top200,MOVE_UP,35557.0
247,Papercuts (feat. Vera Blue),47,2017-01-01,Illy,https://open.spotify.com/track/6vXHaWpqkbuvzID...,Australia,top200,MOVE_DOWN,35532.0
248,Bad Things (with Camila Cabello),48,2017-01-01,Machine Gun Kelly,https://open.spotify.com/track/1PSBzsahR2AKwLJ...,Australia,top200,MOVE_UP,35340.0


In [6]:
spotify_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26173514 entries, 0 to 26173513
Data columns (total 9 columns):
 #   Column   Dtype  
---  ------   -----  
 0   title    object 
 1   rank     int64  
 2   date     object 
 3   artist   object 
 4   url      object 
 5   region   object 
 6   chart    object 
 7   trend    object 
 8   streams  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 1.8+ GB


In [17]:
spotify_df['chart'].unique()

array(['top200', 'viral50'], dtype=object)

In [19]:
spotify_df[spotify_df['chart'] == 'viral50']

,title,rank,date,artist,url,region,chart,trend,streams
12808,Boys,13,2017-08-01,Charli XCX,https://open.spotify.com/track/5o4yGlG0PfeVUa6...,Latvia,viral50,MOVE_UP,NaN
12965,Hold Me,19,2017-08-01,R3HAB,https://open.spotify.com/track/2a9xGmpgNyNz9QV...,Luxembourg,viral50,MOVE_DOWN,NaN
13781,Hola Hola,40,2017-08-01,KARD,https://open.spotify.com/track/3VC1IEz9M1txlMS...,Peru,viral50,NEW_ENTRY,NaN
14042,Cuídate,4,2017-08-01,Zetazen,https://open.spotify.com/track/0WgnbqlHieZmdYa...,Spain,viral50,SAME_POSITION,NaN
14092,Boys,13,2017-08-01,Charli XCX,https://open.spotify.com/track/5o4yGlG0PfeVUa6...,Taiwan,viral50,MOVE_DOWN,NaN
...,...,...,...,...,...,...,...,...,...
26173509,BYE,46,2021-07-31,Jaden,https://open.spotify.com/track/3OUyyDN7EZrL7i0...,Vietnam,viral50,MOVE_UP,NaN
26173510,Pillars,47,2021-07-31,My Anh,https://open.spotify.com/track/6eky30oFiQbHUAT...,Vietnam,viral50,NEW_ENTRY,NaN
26173511,Gái Độc Thân,48,2021-07-31,Tlinh,https://open.spotify.com/track/2klsSb2iTfgDh95...,Vietnam,viral50,MOVE_DOWN,NaN
26173512,Renegade (feat. Taylor Swift),49,2021-07-31,Big Red Machine,https://open.spotify.com/track/1aU1wpYBSpP0M6I...,Vietnam,viral50,MOVE_DOWN,NaN


In [8]:
# CLEANING SPOTIFY
# i just dropped the nulls in title, artist, and streams bc they seem like important info to have
spotify_inter = spotify_df.dropna(subset=['title', 'artist', 'streams'])
spotify_inter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20321897 entries, 0 to 25276073
Data columns (total 9 columns):
 #   Column   Dtype  
---  ------   -----  
 0   title    object 
 1   rank     int64  
 2   date     object 
 3   artist   object 
 4   url      object 
 5   region   object 
 6   chart    object 
 7   trend    object 
 8   streams  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 1.5+ GB


In [9]:
spotify_df['chart'].unique()

array(['top200', 'viral50'], dtype=object)

In [10]:
# this converts the date to day, month, year
spotify_inter['date'] = pd.to_datetime(spotify_inter['date'])

spotify_inter['day'] = spotify_inter['date'].dt.day
spotify_inter['month'] = spotify_inter['date'].dt.month
spotify_inter['year'] = spotify_inter['date'].dt.year

<ipython-input-10-c74f1ccb4acb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spotify_inter['date'] = pd.to_datetime(spotify_inter['date'])
<ipython-input-10-c74f1ccb4acb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spotify_inter['day'] = spotify_inter['date'].dt.day
<ipython-input-10-c74f1ccb4acb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [11]:
# if multiple artists, split into different rows
# only consider US charts
spotify_us = spotify_inter[spotify_inter['region']=='United States']
spotify_us

,title,rank,date,artist,url,region,chart,trend,streams,day,month,year
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0,1,1,2017
8247,Fake Love,2,2017-01-01,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,United States,top200,SAME_POSITION,1180074.0,1,1,2017
8248,Starboy,3,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,1,1,2017
8249,Closer,4,2017-01-01,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0,1,1,2017
8250,Black Beatles,5,2017-01-01,"Rae Sremmurd, Gucci Mane",https://open.spotify.com/track/6fujklziTHa8uoM...,United States,top200,SAME_POSITION,874289.0,1,1,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
25275869,X (feat. Future),196,2018-01-31,"21 Savage, Metro Boomin",https://open.spotify.com/track/6fwdbPMwP1zVStm...,United States,top200,NEW_ENTRY,200771.0,31,1,2018
25275870,Poles 1469,197,2018-01-31,"Trippie Redd, 6ix9ine",https://open.spotify.com/track/4fC5jw1TI59GH1e...,United States,top200,NEW_ENTRY,200072.0,31,1,2018
25275871,Look What You Made Me Do,198,2018-01-31,Taylor Swift,https://open.spotify.com/track/5troof8mcGO3Aaf...,United States,top200,MOVE_DOWN,199702.0,31,1,2018
25275872,Slow Hands,199,2018-01-31,Niall Horan,https://open.spotify.com/track/38yBBH2jacvDxrz...,United States,top200,NEW_ENTRY,198353.0,31,1,2018


In [12]:
spotify_us['artist'] = spotify_us['artist'].str.split(',').apply(lambda x: [i.strip() for i in x])

<ipython-input-12-13da7d15652c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spotify_us['artist'] = spotify_us['artist'].str.split(',').apply(lambda x: [i.strip() for i in x])


In [13]:
spotify_us = spotify_us.explode('artist').reset_index(drop=True)
spotify_us

,title,rank,date,artist,url,region,chart,trend,streams,day,month,year
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0,1,1,2017
1,Fake Love,2,2017-01-01,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,United States,top200,SAME_POSITION,1180074.0,1,1,2017
2,Starboy,3,2017-01-01,The Weeknd,https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,1,1,2017
3,Starboy,3,2017-01-01,Daft Punk,https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,1,1,2017
4,Closer,4,2017-01-01,The Chainsmokers,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0,1,1,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
437233,Poles 1469,197,2018-01-31,Trippie Redd,https://open.spotify.com/track/4fC5jw1TI59GH1e...,United States,top200,NEW_ENTRY,200072.0,31,1,2018
437234,Poles 1469,197,2018-01-31,6ix9ine,https://open.spotify.com/track/4fC5jw1TI59GH1e...,United States,top200,NEW_ENTRY,200072.0,31,1,2018
437235,Look What You Made Me Do,198,2018-01-31,Taylor Swift,https://open.spotify.com/track/5troof8mcGO3Aaf...,United States,top200,MOVE_DOWN,199702.0,31,1,2018
437236,Slow Hands,199,2018-01-31,Niall Horan,https://open.spotify.com/track/38yBBH2jacvDxrz...,United States,top200,NEW_ENTRY,198353.0,31,1,2018


In [12]:
temp = spotify_us[spotify_us['month']!=1]
temp

,title,rank,date,artist,url,region,chart,trend,streams,day,month,year
247,God's Plan,1,2018-03-01,Drake,https://open.spotify.com/track/2XW4DbS6NddZxRP...,United States,top200,SAME_POSITION,3236848.0,1,3,2018
248,Psycho (feat. Ty Dolla $ign),2,2018-03-01,Post Malone,https://open.spotify.com/track/65NwOZqoXny4Jxq...,United States,top200,SAME_POSITION,2877840.0,1,3,2018
249,Look Alive (feat. Drake),3,2018-03-01,BlocBoy JB,https://open.spotify.com/track/4qKcDkK6siZ7Jp1...,United States,top200,SAME_POSITION,1681321.0,1,3,2018
250,All The Stars (with SZA),4,2018-03-01,Kendrick Lamar,https://open.spotify.com/track/3GCdLUSnKSMJhs4...,United States,top200,SAME_POSITION,1634650.0,1,3,2018
251,Stir Fry,5,2018-03-01,Migos,https://open.spotify.com/track/2UVbBKQOdFAekPT...,United States,top200,SAME_POSITION,1253369.0,1,3,2018
...,...,...,...,...,...,...,...,...,...,...,...,...
436413,Criminal,197,2017-12-31,Ozuna,https://open.spotify.com/track/6Za3190Sbw39BBC...,United States,top200,MOVE_DOWN,175880.0,31,12,2017
436414,Galway Girl,198,2017-12-31,Ed Sheeran,https://open.spotify.com/track/0afhq8XCExXpqaz...,United States,top200,MOVE_DOWN,174859.0,31,12,2017
436415,From The D To The A (feat. Lil Yachty),199,2017-12-31,Tee Grizzley,https://open.spotify.com/track/2NyrXRn4tancYPW...,United States,top200,MOVE_DOWN,174168.0,31,12,2017
436416,It Wasn't Me,200,2017-12-31,Shaggy,https://open.spotify.com/track/1lzr43nnXAijIGY...,United States,top200,NEW_ENTRY,171779.0,31,12,2017


In [16]:
spotify_inter['chart'].unique()

array(['top200'], dtype=object)

In [15]:
spotify_inter = spotify_us
spotify_inter.head(50)

,title,rank,date,artist,url,region,chart,trend,streams,day,month,year
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0,1,1,2017
1,Fake Love,2,2017-01-01,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,United States,top200,SAME_POSITION,1180074.0,1,1,2017
2,Starboy,3,2017-01-01,The Weeknd,https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,1,1,2017
3,Starboy,3,2017-01-01,Daft Punk,https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,1,1,2017
4,Closer,4,2017-01-01,The Chainsmokers,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0,1,1,2017
5,Closer,4,2017-01-01,Halsey,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0,1,1,2017
6,Black Beatles,5,2017-01-01,Rae Sremmurd,https://open.spotify.com/track/6fujklziTHa8uoM...,United States,top200,SAME_POSITION,874289.0,1,1,2017
7,Black Beatles,5,2017-01-01,Gucci Mane,https://open.spotify.com/track/6fujklziTHa8uoM...,United States,top200,SAME_POSITION,874289.0,1,1,2017
8,Broccoli (feat. Lil Yachty),6,2017-01-01,Shelley FKA DRAM,https://open.spotify.com/track/7yyRTcZmCiyzzJl...,United States,top200,MOVE_UP,763259.0,1,1,2017
9,One Dance,7,2017-01-01,Drake,https://open.spotify.com/track/1xznGGDReH1oQq0...,United States,top200,MOVE_UP,753150.0,1,1,2017


In [43]:
spotify_inter

,title,rank,date,artist,url,region,chart,trend,streams,day,month,year
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0,1,1,2017
1,Fake Love,2,2017-01-01,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,United States,top200,SAME_POSITION,1180074.0,1,1,2017
2,Starboy,3,2017-01-01,The Weeknd,https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,1,1,2017
3,Starboy,3,2017-01-01,Daft Punk,https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,1,1,2017
4,Closer,4,2017-01-01,The Chainsmokers,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0,1,1,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
437233,Poles 1469,197,2018-01-31,Trippie Redd,https://open.spotify.com/track/4fC5jw1TI59GH1e...,United States,top200,NEW_ENTRY,200072.0,31,1,2018
437234,Poles 1469,197,2018-01-31,6ix9ine,https://open.spotify.com/track/4fC5jw1TI59GH1e...,United States,top200,NEW_ENTRY,200072.0,31,1,2018
437235,Look What You Made Me Do,198,2018-01-31,Taylor Swift,https://open.spotify.com/track/5troof8mcGO3Aaf...,United States,top200,MOVE_DOWN,199702.0,31,1,2018
437236,Slow Hands,199,2018-01-31,Niall Horan,https://open.spotify.com/track/38yBBH2jacvDxrz...,United States,top200,NEW_ENTRY,198353.0,31,1,2018


In [44]:
spotify_inter.isnull().any() # no more null

title      False
rank       False
date       False
artist     False
url        False
region     False
chart      False
trend      False
streams    False
day        False
month      False
year       False
dtype: bool

In [45]:
spotify_inter.head(10)

,title,rank,date,artist,url,region,chart,trend,streams,day,month,year
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0,1,1,2017
1,Fake Love,2,2017-01-01,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,United States,top200,SAME_POSITION,1180074.0,1,1,2017
2,Starboy,3,2017-01-01,The Weeknd,https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,1,1,2017
3,Starboy,3,2017-01-01,Daft Punk,https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,1,1,2017
4,Closer,4,2017-01-01,The Chainsmokers,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0,1,1,2017
5,Closer,4,2017-01-01,Halsey,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0,1,1,2017
6,Black Beatles,5,2017-01-01,Rae Sremmurd,https://open.spotify.com/track/6fujklziTHa8uoM...,United States,top200,SAME_POSITION,874289.0,1,1,2017
7,Black Beatles,5,2017-01-01,Gucci Mane,https://open.spotify.com/track/6fujklziTHa8uoM...,United States,top200,SAME_POSITION,874289.0,1,1,2017
8,Broccoli (feat. Lil Yachty),6,2017-01-01,Shelley FKA DRAM,https://open.spotify.com/track/7yyRTcZmCiyzzJl...,United States,top200,MOVE_UP,763259.0,1,1,2017
9,One Dance,7,2017-01-01,Drake,https://open.spotify.com/track/1xznGGDReH1oQq0...,United States,top200,MOVE_UP,753150.0,1,1,2017


In [46]:
# spotify_df.rename(columns={'rank': 'chart_rank'}, inplace=True)
spotify_inter

,title,rank,date,artist,url,region,chart,trend,streams,day,month,year
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0,1,1,2017
1,Fake Love,2,2017-01-01,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,United States,top200,SAME_POSITION,1180074.0,1,1,2017
2,Starboy,3,2017-01-01,The Weeknd,https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,1,1,2017
3,Starboy,3,2017-01-01,Daft Punk,https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,1,1,2017
4,Closer,4,2017-01-01,The Chainsmokers,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0,1,1,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
437233,Poles 1469,197,2018-01-31,Trippie Redd,https://open.spotify.com/track/4fC5jw1TI59GH1e...,United States,top200,NEW_ENTRY,200072.0,31,1,2018
437234,Poles 1469,197,2018-01-31,6ix9ine,https://open.spotify.com/track/4fC5jw1TI59GH1e...,United States,top200,NEW_ENTRY,200072.0,31,1,2018
437235,Look What You Made Me Do,198,2018-01-31,Taylor Swift,https://open.spotify.com/track/5troof8mcGO3Aaf...,United States,top200,MOVE_DOWN,199702.0,31,1,2018
437236,Slow Hands,199,2018-01-31,Niall Horan,https://open.spotify.com/track/38yBBH2jacvDxrz...,United States,top200,NEW_ENTRY,198353.0,31,1,2018


In [47]:
spotify_inter['chart'].unique()

array(['top200'], dtype=object)

In [59]:
spotify_df1 = spotify_inter[['rank', 'url', 'artist', 'trend', 'streams', 'day', 'month', 'year']]
spotify_df2 = spotify_inter[['url', 'title', 'artist']]

In [60]:
spotify_df1.rename(columns={'rank': 'chart_rank'}, inplace=True)
spotify_df1

<ipython-input-60-451d34aa3c74>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spotify_df1.rename(columns={'rank': 'chart_rank'}, inplace=True)


,chart_rank,url,artist,trend,streams,day,month,year
0,1,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,Migos,SAME_POSITION,1371493.0,1,1,2017
1,2,https://open.spotify.com/track/343YBumqHu19cGo...,Drake,SAME_POSITION,1180074.0,1,1,2017
2,3,https://open.spotify.com/track/5aAx2yezTd8zXrk...,The Weeknd,SAME_POSITION,1064351.0,1,1,2017
3,3,https://open.spotify.com/track/5aAx2yezTd8zXrk...,Daft Punk,SAME_POSITION,1064351.0,1,1,2017
4,4,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,The Chainsmokers,SAME_POSITION,1010492.0,1,1,2017
...,...,...,...,...,...,...,...,...
437233,197,https://open.spotify.com/track/4fC5jw1TI59GH1e...,Trippie Redd,NEW_ENTRY,200072.0,31,1,2018
437234,197,https://open.spotify.com/track/4fC5jw1TI59GH1e...,6ix9ine,NEW_ENTRY,200072.0,31,1,2018
437235,198,https://open.spotify.com/track/5troof8mcGO3Aaf...,Taylor Swift,MOVE_DOWN,199702.0,31,1,2018
437236,199,https://open.spotify.com/track/38yBBH2jacvDxrz...,Niall Horan,NEW_ENTRY,198353.0,31,1,2018


In [61]:
spotify_df1 = spotify_df1.drop_duplicates()

In [64]:
duplicates = spotify_df1.duplicated(['chart_rank', 'day', 'month', 'year'], keep=False)  # 'keep=False' marks all duplicates as True

# Display duplicate rows
print(spotify_df1[duplicates])

        chart_rank                                                url  \
2                3  https://open.spotify.com/track/5aAx2yezTd8zXrk...   
3                3  https://open.spotify.com/track/5aAx2yezTd8zXrk...   
4                4  https://open.spotify.com/track/7BKLCZ1jbUBVqRi...   
5                4  https://open.spotify.com/track/7BKLCZ1jbUBVqRi...   
6                5  https://open.spotify.com/track/6fujklziTHa8uoM...   
...            ...                                                ...   
437221         186  https://open.spotify.com/track/6yr8GiTHWvFfi4o...   
437231         196  https://open.spotify.com/track/6fwdbPMwP1zVStm...   
437232         196  https://open.spotify.com/track/6fwdbPMwP1zVStm...   
437233         197  https://open.spotify.com/track/4fC5jw1TI59GH1e...   
437234         197  https://open.spotify.com/track/4fC5jw1TI59GH1e...   

                  artist          trend    streams  day  month  year  
2             The Weeknd  SAME_POSITION  1064351.0  

In [51]:
spotify_df2 = spotify_df2.drop_duplicates()

In [53]:
duplicates = spotify_df2.duplicated(['url'], keep=False)  # 'keep=False' marks all duplicates as True

# Display duplicate rows
print(spotify_df2[duplicates])

                                                      url  \
2       https://open.spotify.com/track/5aAx2yezTd8zXrk...   
3       https://open.spotify.com/track/5aAx2yezTd8zXrk...   
4       https://open.spotify.com/track/7BKLCZ1jbUBVqRi...   
5       https://open.spotify.com/track/7BKLCZ1jbUBVqRi...   
6       https://open.spotify.com/track/6fujklziTHa8uoM...   
...                                                   ...   
436091  https://open.spotify.com/track/62rchuhJrm5JSlI...   
436092  https://open.spotify.com/track/62rchuhJrm5JSlI...   
436093  https://open.spotify.com/track/62rchuhJrm5JSlI...   
436129  https://open.spotify.com/track/15HMh4yxdf4wyxS...   
436130  https://open.spotify.com/track/15HMh4yxdf4wyxS...   

                                               title            artist  
2                                            Starboy        The Weeknd  
3                                            Starboy         Daft Punk  
4                                             Cl

In [85]:
spotify_df2 = spotify_df2.drop(index=371427)
spotify_df2 = spotify_df2.drop(index=436129)
spotify_df2 = spotify_df2.drop(index=371411)

# Cleaning Ticketmaster df

In [88]:
# snapshot of ticketmaster
ticket_df.head(50)

,event_url,event_id,event_category,event_subcategory,title,datetime_utc,datetime_local,name,address,city,state,country,postal_code,formatted_address,day,month,year,time
0,https://www.ticketmaster.com/absinthe-las-vega...,39005D610179ADFF,ARTS & THEATRE,PERFORMANCE ART,Absinthe,2023/08/02 05:00:00,2023-08-01 22:00:00,The Spiegeltent at Caesars Palace,3570 las vegas blvd south,Las Vegas,NV,US,89109,"3570 Las Vegas Blvd S, Las Vegas, NV 89109, US",1,8,2023,22:00:00
1,https://www.ticketmaster.com/ally-venable-191-...,19005EC2B8982BC8,MUSIC,ROCK,Ally Venable @ 191 Toole,2023/08/02 03:00:00,2023-08-01 20:00:00,191 Toole,191 e toole ave,Tucson,AZ,US,85701,"191 E Toole Ave, Tucson, AZ 85701, US",1,8,2023,20:00:00
2,https://www.ticketmaster.com/des-moines-storyt...,06005D77AE02514A,MISCELLANEOUS,LECTURE/SEMINAR,Des Moines Storytellers Project Season Package,2023/02/15 01:00:00,2023-02-14 19:00:00,Hoyt Sherman Place,1501 woodland avenue,Des Moines,IA,US,50309,"1501 Woodland Ave, Des Moines, IA 50309, US",14,2,2023,19:00:00
3,https://www.ticketmaster.com/fantasy-las-vegas...,2E005D34F9EA55A8,ARTS & THEATRE,PERFORMANCE ART,Fantasy,2023/08/02 05:30:00,2023-08-01 22:30:00,Atrium Showroom at Luxor Hotel and Casino Las ...,3900 las vegas blvd south,Las Vegas,NV,US,89109,"3900 Las Vegas Blvd S, Las Vegas, NV 89119, US",1,8,2023,22:30:00
4,https://www.ticketmaster.com/fred-armisen-come...,0B005ED5E51A297E,ARTS & THEATRE,COMEDY,Fred Armisen - Comedy For Musicians But Everyo...,2023/08/02 03:00:00,2023-08-01 20:00:00,Beverly O'Neill Theater - Long Beach Conv and ...,300 e ocean bl,Long Beach,CA,US,90802,"300 E Ocean Blvd, Long Beach, CA 90802, US",1,8,2023,20:00:00
5,https://www.ticketmaster.com/john-caparulo-at-...,39005E57CDE896D6,ARTS & THEATRE,COMEDY,John Caparulo At Jimmy Kimmel's Comedy Club,2023/08/02 04:30:00,2023-08-01 21:30:00,Jimmy Kimmel's Comedy Club,3545 s las vegas blvd,Las Vegas,NV,US,89109,"3545 S Las Vegas Blvd, Las Vegas, NV 89109, US",1,8,2023,21:30:00
6,https://www.ticketmaster.com/lany-los-angeles-...,09005EC1D06839AA,MUSIC,ROCK,LANY,2023/08/02 03:00:00,2023-08-01 20:00:00,The Bellwether,333 s. boylston st.,Los Angeles,CA,US,90025,"333 S Boylston St, Los Angeles, CA 90017, US",1,8,2023,20:00:00
7,https://www.ticketmaster.com/laugh-factory-las...,17005D63B60156FD,ARTS & THEATRE,COMEDY,Laugh Factory Las Vegas,2023/08/02 05:30:00,2023-08-01 22:30:00,Laugh Factory Las Vegas,3801 s las vegas blvd,Las Vegas,NV,US,89109,"3801 S Las Vegas Blvd, Las Vegas, NV 89109, US",1,8,2023,22:30:00
8,https://www.ticketmaster.com/rise-los-angeles-...,09005EFF872A12B5,MUSIC,ROCK,Rise,2023/08/02 03:00:00,2023-08-01 20:00:00,The Moroccan Lounge,901 e 1st street,Los Angeles,CA,US,90012,"901 1st St, Los Angeles, CA 90012, US",1,8,2023,20:00:00
9,https://www.ticketmaster.com/the-ultimate-4d-e...,2E005E7BD6E21EC3,FILM,FAMILY,The Ultimate 4d Experience,2023/08/01 19:00:00,2023-08-01 12:00:00,Excalibur Hotel and Casino,3850 las vegas blvd south,Las Vegas,NV,US,89109,"3850 Las Vegas Blvd S, Las Vegas, NV 89109, US",1,8,2023,12:00:00


In [89]:
ticket_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80053 entries, 0 to 80052
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   event_url          80053 non-null  object
 1   event_id           80053 non-null  object
 2   event_category     80053 non-null  object
 3   event_subcategory  76242 non-null  object
 4   title              80053 non-null  object
 5   datetime_utc       80053 non-null  object
 6   datetime_local     80053 non-null  object
 7   name               80053 non-null  object
 8   address            78460 non-null  object
 9   city               80053 non-null  object
 10  state              80053 non-null  object
 11  country            80053 non-null  object
 12  postal_code        80017 non-null  object
 13  formatted_address  71609 non-null  object
 14  day                80053 non-null  int64 
 15  month              80053 non-null  int64 
 16  year               80053 non-null  int64

In [93]:
# CLEANING TICKETMASTER
ticket_df.isnull().any() # since none are null, i won't drop

event_url            False
event_id             False
event_category       False
event_subcategory    False
title                False
datetime_utc         False
datetime_local       False
name                 False
address               True
city                 False
state                False
country              False
postal_code           True
formatted_address     True
day                  False
month                False
year                 False
time                 False
dtype: bool

In [91]:
ticket_df = ticket_df[ticket_df['event_category'] == 'MUSIC']

In [92]:
ticket_df

,event_url,event_id,event_category,event_subcategory,title,datetime_utc,datetime_local,name,address,city,state,country,postal_code,formatted_address,day,month,year,time
1,https://www.ticketmaster.com/ally-venable-191-...,19005EC2B8982BC8,MUSIC,ROCK,Ally Venable @ 191 Toole,2023/08/02 03:00:00,2023-08-01 20:00:00,191 Toole,191 e toole ave,Tucson,AZ,US,85701,"191 E Toole Ave, Tucson, AZ 85701, US",1,8,2023,20:00:00
6,https://www.ticketmaster.com/lany-los-angeles-...,09005EC1D06839AA,MUSIC,ROCK,LANY,2023/08/02 03:00:00,2023-08-01 20:00:00,The Bellwether,333 s. boylston st.,Los Angeles,CA,US,90025,"333 S Boylston St, Los Angeles, CA 90017, US",1,8,2023,20:00:00
8,https://www.ticketmaster.com/rise-los-angeles-...,09005EFF872A12B5,MUSIC,ROCK,Rise,2023/08/02 03:00:00,2023-08-01 20:00:00,The Moroccan Lounge,901 e 1st street,Los Angeles,CA,US,90012,"901 1st St, Los Angeles, CA 90012, US",1,8,2023,20:00:00
11,https://www.ticketmaster.com/collective-soul-s...,0A005E64D30D328A,MUSIC,ROCK,Collective Soul,2023/08/03 02:30:00,2023-08-02 19:30:00,Humphreys Concerts By the Bay,2241 shelter island dr.,San Diego,CA,US,92106,"2241 Shelter Island Dr, San Diego, CA 92106, US",2,8,2023,19:30:00
21,https://www.ticketmaster.com/counting-crows-ba...,2D005E63C2FC71DC,MUSIC,ROCK,Counting Crows: Banshee Season Tour with Dashb...,2023/08/02 23:00:00,2023-08-02 19:00:00,Credit One Stadium,161 seven farms drive,Charleston,SC,US,29492,"161 Seven Farms Drive, Daniel Island, SC 29492...",2,8,2023,19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80047,https://www.ticketmaster.com/brit-floyd-fort-w...,05005E66D3264757,MUSIC,UNDEFINED,Brit Floyd,2025/05/12 00:00:00,2025-05-11 20:00:00,FOELLINGER THEATRE,3411 sherman blvd.,Fort Wayne,IN,US,46802,"3411 Sherman Blvd, Fort Wayne, IN 46808, US",11,5,2025,20:00:00
80048,https://www.ticketmaster.com/jimmie-allen-hend...,17005E34AB3E8572,MUSIC,COUNTRY,Jimmie Allen,2025/05/18 03:00:00,2025-05-17 20:00:00,Sunset Station Outdoor Amphitheater,1301 west sunset road,Henderson,NV,US,89014,"1301 W Sunset Rd, Henderson, NV 89014, US",17,5,2025,20:00:00
80049,https://www.ticketmaster.com/event/Z7r9jZ1Ad-zON,Z7r9jZ1Ad-zON,MUSIC,OTHER,Babytron,2025/05/07 23:00:00,2025-05-07 19:00:00,Webster Theatre,31 webster st,Hartford,CT,US,06114,"31 Webster St, Hartford, CT 06114, US",7,5,2025,19:00:00
80050,https://www.ticketmaster.com/blessd-blessd-cor...,17005E7BCDF965EC,MUSIC,HIP-HOP/RAP,Blessd - Blessd Corner,2025/05/08 03:00:00,2025-05-07 20:00:00,The Cosmopolitan of Las Vegas,3708 las vegas boulevard south,Las Vegas,NV,US,89109,"3708 Las Vegas Blvd S, Las Vegas, NV 89109, US",7,5,2025,20:00:00


In [ ]:
duplicate_rows = ticket_df.duplicated()
print(ticket_df[duplicate_rows])

Empty DataFrame
Columns: [event_id, name, state, city, title, event_category, event_subcategory, datetime_utc, datetime_local, address, formatted_address, postal_code, day, month, year, time, event_url, artist]
Index: []


In [96]:
ticket_df.isnull().any()

event_url            False
event_id             False
event_category       False
event_subcategory    False
title                False
datetime_utc         False
datetime_local       False
name                 False
address               True
city                 False
state                False
country              False
postal_code           True
formatted_address     True
day                  False
month                False
year                 False
time                 False
dtype: bool

In [95]:
ticket_df = ticket_df[ticket_df['event_category']=='MUSIC']

In [ ]:
ticket_df = ticket_df.dropna(subset=['artist'])

In [ ]:
ticket_df

,event_id,name,state,city,title,event_category,event_subcategory,datetime_utc,datetime_local,address,formatted_address,postal_code,day,month,year,time,event_url,artist
2,00005855B4558156,Bergen Performing Arts Center,NJ,Englewood,Straight No Chaser,MUSIC,ROCK,2024-05-03 00:00:00,2024-05-03 00:00:00,30 n. van brunt street,"30 N Van Brunt St, Englewood, NJ 07631, US",7631,3,5,2024,19:30:00,https://www.ticketmaster.com/straight-no-chase...,Straight No Chaser
3,00005AC99F0C1FD5,Knitting Factory,NY,Brooklyn,Blitzkid - Escape the Grave Tour,MUSIC,ROCK,2024-01-10 00:00:00,2024-01-09 00:00:00,361 metropolitan avenue,"361 Metropolitan Ave, Brooklyn, NY 11211, US",11211,9,1,2024,19:30:00,https://www.ticketmaster.com/blitzkid-escape-t...,Blitzkid
4,00005AE6BE623FE3,Sony Hall,NY,New York,Eric Johnson's Treasure Tour,MUSIC,ROCK,2023-09-20 00:00:00,2023-09-19 00:00:00,235 w 46th street,"235 W 46th St, New York, NY 10036, US",10036,19,9,2023,20:00:00,https://www.ticketmaster.com/eric-johnsons-tre...,Eric Johnson
8,00005C2CEDBA5915,Glenville Spot,NY,Glenville,Test for FVE - no manifest file,MUSIC,ROCK,2024-02-16 00:00:00,2024-02-15 00:00:00,123 street st,"Glenville, NY 12010, US",12302,15,2,2024,19:30:00,https://www.ticketmaster.com/test-for-fve-no-m...,Seekae
9,00005C3BDC364EA5,The Palace Theatre Albany,NY,Albany,Gov't Mule,MUSIC,ROCK,2024-03-29 00:00:00,2024-03-28 00:00:00,19 clinton ave,"19 Clinton Ave, Albany, NY 12207, US",12207,28,3,2024,20:00:00,https://www.ticketmaster.com/govt-mule-albany-...,Gov't Mule
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71604,Z7r9jZ1AdxZx3,Cook's Garage,TX,Lubbock,Rick Springfield,MUSIC,POP,2023-08-23 00:00:00,2023-08-22 00:00:00,11002 us-87 d,"11002 US-87 Lubbock, TX 79423, US",79423,22,8,2023,19:30:00,https://www.ticketmaster.com/event/Z7r9jZ1AdxZx3,Rick Springfield
71605,Z7r9jZ1AdxZx4,Roseland Theater,OR,Portland,Tom Odell,MUSIC,COUNTRY,2023-10-27 00:00:00,2023-10-26 00:00:00,6 nw 6th avenue,"6 NW 6th Ave, Portland, OR 97209, US",97209,26,10,2023,20:00:00,https://www.ticketmaster.com/event/Z7r9jZ1AdxZx4,Tom Odell
71606,Z7r9jZ1AdxZxa,The Commonwealth Room,UT,Salt Lake City,Tom Odell,MUSIC,COUNTRY,2023-10-22 00:00:00,2023-10-21 00:00:00,195 w 2100 s,"195 W 2100 S Expy, Salt Lake City, UT 84115, US",84115,21,10,2023,20:00:00,https://www.ticketmaster.com/event/Z7r9jZ1AdxZxa,Tom Odell
71607,Z7r9jZ1AdxZxF,First Avenue,MN,Minneapolis,Tom Odell,MUSIC,COUNTRY,2023-10-19 00:00:00,2023-10-18 00:00:00,701 1st ave n,"701 N 1st Ave, Minneapolis, MN 55403, US",55403,18,10,2023,20:00:00,https://www.ticketmaster.com/event/Z7r9jZ1AdxZxF,Tom Odell


In [ ]:
ticket_df = ticket_df.drop_duplicates()

In [ ]:
duplicate_rows = ticket_df.duplicated()
print(ticket_df[duplicate_rows])

Empty DataFrame
Columns: [event_id, name, state, city, title, event_category, event_subcategory, datetime_utc, datetime_local, address, formatted_address, postal_code, day, month, year, time, event_url, artist]
Index: []


In [ ]:
ticket_inter = ticket_df
# this converts the date to day, month, year
ticket_inter['datetime_local'] = pd.to_datetime(ticket_inter['datetime_local'])

ticket_inter['day'] = ticket_inter['datetime_local'].dt.day
ticket_inter['month'] = ticket_inter['datetime_local'].dt.month
ticket_inter['year'] = ticket_inter['datetime_local'].dt.year
ticket_inter['time'] = ticket_inter['datetime_local'].dt.time

In [ ]:
ticket_music = ticket_inter[ticket_inter['event_category']=='MUSIC']
ticket_music

,event_id,name,state,city,title,event_category,event_subcategory,datetime_utc,datetime_local,address,formatted_address,postal_code,day,month,year,time,event_url,artist
2,00005855B4558156,Bergen Performing Arts Center,NJ,Englewood,Straight No Chaser,MUSIC,ROCK,2024-05-03 00:00:00,2024-05-03,30 n. van brunt street,"30 N Van Brunt St, Englewood, NJ 07631, US",7631,3,5,2024,00:00:00,https://www.ticketmaster.com/straight-no-chase...,Straight No Chaser
3,00005AC99F0C1FD5,Knitting Factory,NY,Brooklyn,Blitzkid - Escape the Grave Tour,MUSIC,ROCK,2024-01-10 00:00:00,2024-01-09,361 metropolitan avenue,"361 Metropolitan Ave, Brooklyn, NY 11211, US",11211,9,1,2024,00:00:00,https://www.ticketmaster.com/blitzkid-escape-t...,Blitzkid
4,00005AE6BE623FE3,Sony Hall,NY,New York,Eric Johnson's Treasure Tour,MUSIC,ROCK,2023-09-20 00:00:00,2023-09-19,235 w 46th street,"235 W 46th St, New York, NY 10036, US",10036,19,9,2023,00:00:00,https://www.ticketmaster.com/eric-johnsons-tre...,Eric Johnson
8,00005C2CEDBA5915,Glenville Spot,NY,Glenville,Test for FVE - no manifest file,MUSIC,ROCK,2024-02-16 00:00:00,2024-02-15,123 street st,"Glenville, NY 12010, US",12302,15,2,2024,00:00:00,https://www.ticketmaster.com/test-for-fve-no-m...,Seekae
9,00005C3BDC364EA5,The Palace Theatre Albany,NY,Albany,Gov't Mule,MUSIC,ROCK,2024-03-29 00:00:00,2024-03-28,19 clinton ave,"19 Clinton Ave, Albany, NY 12207, US",12207,28,3,2024,00:00:00,https://www.ticketmaster.com/govt-mule-albany-...,Gov't Mule
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71604,Z7r9jZ1AdxZx3,Cook's Garage,TX,Lubbock,Rick Springfield,MUSIC,POP,2023-08-23 00:00:00,2023-08-22,11002 us-87 d,"11002 US-87 Lubbock, TX 79423, US",79423,22,8,2023,00:00:00,https://www.ticketmaster.com/event/Z7r9jZ1AdxZx3,Rick Springfield
71605,Z7r9jZ1AdxZx4,Roseland Theater,OR,Portland,Tom Odell,MUSIC,COUNTRY,2023-10-27 00:00:00,2023-10-26,6 nw 6th avenue,"6 NW 6th Ave, Portland, OR 97209, US",97209,26,10,2023,00:00:00,https://www.ticketmaster.com/event/Z7r9jZ1AdxZx4,Tom Odell
71606,Z7r9jZ1AdxZxa,The Commonwealth Room,UT,Salt Lake City,Tom Odell,MUSIC,COUNTRY,2023-10-22 00:00:00,2023-10-21,195 w 2100 s,"195 W 2100 S Expy, Salt Lake City, UT 84115, US",84115,21,10,2023,00:00:00,https://www.ticketmaster.com/event/Z7r9jZ1AdxZxa,Tom Odell
71607,Z7r9jZ1AdxZxF,First Avenue,MN,Minneapolis,Tom Odell,MUSIC,COUNTRY,2023-10-19 00:00:00,2023-10-18,701 1st ave n,"701 N 1st Ave, Minneapolis, MN 55403, US",55403,18,10,2023,00:00:00,https://www.ticketmaster.com/event/Z7r9jZ1AdxZxF,Tom Odell


In [ ]:
ticket = ticket_music[ticket_music['title'].str.contains('Bruno Mars')]
ticket

,event_id,name,state,city,title,event_category,event_subcategory,datetime_utc,datetime_local,address,formatted_address,postal_code,day,month,year,time,event_url,artist
20815,0F005EF9B4EA2019,Emerald Queen Casino,WA,Tacoma,24K Magic - The #1 Tribute to Bruno Mars,MUSIC,UNDEFINED,2023-09-22 00:00:00,2023-09-21,2920 east r street,"2920 E R St, Tacoma, WA 98404, US",98404,21,9,2023,00:00:00,https://www.ticketmaster.com/24k-magic-the-1-t...,Harmony River
38627,2E005EB7E7DD3530,Dolby Live,NV,Las Vegas,Bruno Mars Las Vegas,MUSIC,ROCK,2023-08-26 00:00:00,2023-08-25,3770 s. las vegas blvd,"3770 S Las Vegas Blvd, Las Vegas, NV 89109, US",89109,25,8,2023,00:00:00,https://www.ticketmaster.com/bruno-mars-las-ve...,Bruno Mars
38628,2E005EB7E7E03532,Dolby Live,NV,Las Vegas,Bruno Mars Las Vegas,MUSIC,ROCK,2023-08-27 00:00:00,2023-08-26,3770 s. las vegas blvd,"3770 S Las Vegas Blvd, Las Vegas, NV 89109, US",89109,26,8,2023,00:00:00,https://www.ticketmaster.com/bruno-mars-las-ve...,Bruno Mars


In [ ]:
ticket_inter.head(50)

,event_id,name,state,city,title,event_category,event_subcategory,datetime_utc,datetime_local,address,formatted_address,postal_code,day,month,year,time,event_url,artist
2,00005855B4558156,Bergen Performing Arts Center,NJ,Englewood,Straight No Chaser,MUSIC,ROCK,2024-05-03 00:00:00,2024-05-03,30 n. van brunt street,"30 N Van Brunt St, Englewood, NJ 07631, US",7631,3,5,2024,00:00:00,https://www.ticketmaster.com/straight-no-chase...,Straight No Chaser
3,00005AC99F0C1FD5,Knitting Factory,NY,Brooklyn,Blitzkid - Escape the Grave Tour,MUSIC,ROCK,2024-01-10 00:00:00,2024-01-09,361 metropolitan avenue,"361 Metropolitan Ave, Brooklyn, NY 11211, US",11211,9,1,2024,00:00:00,https://www.ticketmaster.com/blitzkid-escape-t...,Blitzkid
4,00005AE6BE623FE3,Sony Hall,NY,New York,Eric Johnson's Treasure Tour,MUSIC,ROCK,2023-09-20 00:00:00,2023-09-19,235 w 46th street,"235 W 46th St, New York, NY 10036, US",10036,19,9,2023,00:00:00,https://www.ticketmaster.com/eric-johnsons-tre...,Eric Johnson
8,00005C2CEDBA5915,Glenville Spot,NY,Glenville,Test for FVE - no manifest file,MUSIC,ROCK,2024-02-16 00:00:00,2024-02-15,123 street st,"Glenville, NY 12010, US",12302,15,2,2024,00:00:00,https://www.ticketmaster.com/test-for-fve-no-m...,Seekae
9,00005C3BDC364EA5,The Palace Theatre Albany,NY,Albany,Gov't Mule,MUSIC,ROCK,2024-03-29 00:00:00,2024-03-28,19 clinton ave,"19 Clinton Ave, Albany, NY 12207, US",12207,28,3,2024,00:00:00,https://www.ticketmaster.com/govt-mule-albany-...,Gov't Mule
10,00005C5899CD29C7,New Jersey Performing Arts Center,NJ,Newark,Silkroad Ensemble with Rhiannon Giddens,MUSIC,WORLD,2024-07-10 00:00:00,2024-07-09,1 center street,"1 Center St, Newark, NJ 07102, US",7102,9,7,2024,00:00:00,https://www.ticketmaster.com/silkroad-ensemble...,Silkroad Ensemble
12,00005C8FAB5E959D,Shea's Performing Arts Center,NY,Buffalo,Lady A - Request Line Tour,MUSIC,COUNTRY,2023-08-25 00:00:00,2023-08-24,646 main street,"646 Main St, Buffalo, NY 14202, US",14202,24,8,2023,00:00:00,https://www.ticketmaster.com/lady-a-request-li...,Lady A
14,00005C9C8D0A276B,The Oncenter Crouse Hinds Theater,NY,Syracuse,Zach Williams A Hundred Highways Tour,MUSIC,ROCK,2023-10-27 00:00:00,2023-10-27,411 montgomery street,"411 Montgomery St, Syracuse, NY 13202, US",13202,27,10,2023,00:00:00,https://www.ticketmaster.com/zach-williams-a-h...,Zach Williams
15,00005CA3D6327059,Kodak Center,NY,Rochester,Zach Williams A Hundred Highways Tour,MUSIC,ROCK,2023-10-28 00:00:00,2023-10-28,200 west ridge rd.,"200 W Ridge Rd, Rochester, NY 14615, US",14604,28,10,2023,00:00:00,https://www.ticketmaster.com/zach-williams-a-h...,Zach Williams
17,00005CEB84CB11D8,MVP Arena,NY,Albany,Bruce Springsteen and The E Street Band 2023 Tour,MUSIC,ROCK,2023-09-19 00:00:00,2023-09-19,51 south pearl street,"51 S Pearl St, Albany, NY 12207, US",12207,19,9,2023,00:00:00,https://www.ticketmaster.com/bruce-springsteen...,Bruce Springsteen


In [ ]:
ticket_inter = ticket_inter.drop(columns=['datetime_utc', 'datetime_local'])
ticket_inter

,event_id,name,state,city,title,event_category,event_subcategory,address,formatted_address,postal_code,day,month,year,time,event_url,artist
2,00005855B4558156,Bergen Performing Arts Center,NJ,Englewood,Straight No Chaser,MUSIC,ROCK,30 n. van brunt street,"30 N Van Brunt St, Englewood, NJ 07631, US",7631,3,5,2024,00:00:00,https://www.ticketmaster.com/straight-no-chase...,Straight No Chaser
3,00005AC99F0C1FD5,Knitting Factory,NY,Brooklyn,Blitzkid - Escape the Grave Tour,MUSIC,ROCK,361 metropolitan avenue,"361 Metropolitan Ave, Brooklyn, NY 11211, US",11211,9,1,2024,00:00:00,https://www.ticketmaster.com/blitzkid-escape-t...,Blitzkid
4,00005AE6BE623FE3,Sony Hall,NY,New York,Eric Johnson's Treasure Tour,MUSIC,ROCK,235 w 46th street,"235 W 46th St, New York, NY 10036, US",10036,19,9,2023,00:00:00,https://www.ticketmaster.com/eric-johnsons-tre...,Eric Johnson
8,00005C2CEDBA5915,Glenville Spot,NY,Glenville,Test for FVE - no manifest file,MUSIC,ROCK,123 street st,"Glenville, NY 12010, US",12302,15,2,2024,00:00:00,https://www.ticketmaster.com/test-for-fve-no-m...,Seekae
9,00005C3BDC364EA5,The Palace Theatre Albany,NY,Albany,Gov't Mule,MUSIC,ROCK,19 clinton ave,"19 Clinton Ave, Albany, NY 12207, US",12207,28,3,2024,00:00:00,https://www.ticketmaster.com/govt-mule-albany-...,Gov't Mule
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71604,Z7r9jZ1AdxZx3,Cook's Garage,TX,Lubbock,Rick Springfield,MUSIC,POP,11002 us-87 d,"11002 US-87 Lubbock, TX 79423, US",79423,22,8,2023,00:00:00,https://www.ticketmaster.com/event/Z7r9jZ1AdxZx3,Rick Springfield
71605,Z7r9jZ1AdxZx4,Roseland Theater,OR,Portland,Tom Odell,MUSIC,COUNTRY,6 nw 6th avenue,"6 NW 6th Ave, Portland, OR 97209, US",97209,26,10,2023,00:00:00,https://www.ticketmaster.com/event/Z7r9jZ1AdxZx4,Tom Odell
71606,Z7r9jZ1AdxZxa,The Commonwealth Room,UT,Salt Lake City,Tom Odell,MUSIC,COUNTRY,195 w 2100 s,"195 W 2100 S Expy, Salt Lake City, UT 84115, US",84115,21,10,2023,00:00:00,https://www.ticketmaster.com/event/Z7r9jZ1AdxZxa,Tom Odell
71607,Z7r9jZ1AdxZxF,First Avenue,MN,Minneapolis,Tom Odell,MUSIC,COUNTRY,701 1st ave n,"701 N 1st Ave, Minneapolis, MN 55403, US",55403,18,10,2023,00:00:00,https://www.ticketmaster.com/event/Z7r9jZ1AdxZxF,Tom Odell


In [ ]:
duplicates = ticket_inter.duplicated('event_id', keep=False)  # 'keep=False' marks all duplicates as True

# Display duplicate rows
print(ticket_inter[duplicates])

Empty DataFrame
Columns: [event_id, name, state, city, title, event_category, event_subcategory, address, formatted_address, postal_code, day, month, year, time, event_url, artist]
Index: []


In [ ]:
ticket_df1 = ticket_inter[['event_id', 'name', 'title', 'event_category', 'event_subcategory', 'formatted_address', 'day', 'month', 'year', 'time', 'event_url', 'artist']]
ticket_df2 = ticket_inter[['formatted_address', 'state', 'city']]

In [ ]:
ticket_df2 = ticket_df2.drop_duplicates()

In [ ]:
duplicates = ticket_df2.duplicated('formatted_address', keep=False)  # 'keep=False' marks all duplicates as True

# Display duplicate rows
print(ticket_df2[duplicates])

                                    formatted_address state           city
15747    777 S Resort Dr, Valley Center, CA 92082, US    CA  Valley Center
16024    777 S Resort Dr, Valley Center, CA 92082, US    CA         Funner
16298  2301 N Highland Ave, Los Angeles, CA 90068, US    CA      Hollywood
60959    444 Mt Rushmore Rd, Rapid City, SD 57701, US    SD     Rapid City
63242    444 Mt Rushmore Rd, Rapid City, SD 57701, US    SD      Rapid Ciy
69031  2301 N Highland Ave, Los Angeles, CA 90068, US    CA    Los Angeles


In [ ]:
ticket_df2 = ticket_df2.drop(index=16024)
ticket_df2 = ticket_df2.drop(index=16298)
ticket_df2 = ticket_df2.drop(index=63242)

In [ ]:
duplicates = ticket_df2.duplicated('formatted_address', keep=False)  # 'keep=False' marks all duplicates as True

# Display duplicate rows
print(ticket_df2[duplicates])

Empty DataFrame
Columns: [formatted_address, state, city]
Index: []


In [ ]:
duplicates = ticket_df1.duplicated()  # 'keep=False' marks all duplicates as True

# Display duplicate rows
print(ticket_df1[duplicates])

Empty DataFrame
Columns: [event_id, name, title, event_category, event_subcategory, formatted_address, day, month, year, time, event_url, artist]
Index: []


In [ ]:
duplicates = ticket_df1.duplicated('event_id', keep=False)  # 'keep=False' marks all duplicates as True

# Display duplicate rows
print(ticket_df1[duplicates])

Empty DataFrame
Columns: [event_id, name, title, event_category, event_subcategory, formatted_address, day, month, year, time, event_url, artist]
Index: []


In [ ]:
# Filter rows where 'event_id' starts with 'Z7r9jZ1Ad-Vbe'
filtered_df = ticket_df1[ticket_df1['event_id'].str.startswith('Z7r9jZ1Ad')]

# Show the filtered DataFrame
print(filtered_df)

            event_id                         name  \
46203  Z7r9jZ1Ad_-14  Champlain Valley Exposition   
46204  Z7r9jZ1Ad_-7M       Plymouth Memorial Hall   
46281  Z7r9jZ1Ad_0r6           INTRUST Bank Arena   
46345  Z7r9jZ1Ad_4bO       Riverside Theatre - WI   
46559  Z7r9jZ1Ad_7Fs             White Eagle Hall   
...              ...                          ...   
71604  Z7r9jZ1AdxZx3                Cook's Garage   
71605  Z7r9jZ1AdxZx4             Roseland Theater   
71606  Z7r9jZ1AdxZxa        The Commonwealth Room   
71607  Z7r9jZ1AdxZxF                 First Avenue   
71608  Z7r9jZ1AdxZxk               Gothic Theatre   

                                 title event_category event_subcategory  \
46203                      Riley Green          MUSIC             OTHER   
46204                        Amy Grant          MUSIC               POP   
46281  Thomas Rhett: Home Team Tour 23          MUSIC           COUNTRY   
46345                   The Piano Guys          MUSIC         C

In [ ]:
print(ticket_df1)

               event_id                           name  \
2      00005855B4558156  Bergen Performing Arts Center   
3      00005AC99F0C1FD5               Knitting Factory   
4      00005AE6BE623FE3                      Sony Hall   
8      00005C2CEDBA5915                 Glenville Spot   
9      00005C3BDC364EA5      The Palace Theatre Albany   
...                 ...                            ...   
71604     Z7r9jZ1AdxZx3                  Cook's Garage   
71605     Z7r9jZ1AdxZx4               Roseland Theater   
71606     Z7r9jZ1AdxZxa          The Commonwealth Room   
71607     Z7r9jZ1AdxZxF                   First Avenue   
71608     Z7r9jZ1AdxZxk                 Gothic Theatre   

                                  title event_category event_subcategory  \
2                    Straight No Chaser          MUSIC              ROCK   
3      Blitzkid - Escape the Grave Tour          MUSIC              ROCK   
4          Eric Johnson's Treasure Tour          MUSIC              ROCK   

In [ ]:
# Filter rows where 'event_id' starts with 'Z7r9jZ1Ad-Vbe' and drop duplicates
distinct_starting_with = ticket_df1[ticket_df1['event_id'].str.startswith('Z7r9jZ1Ad')].drop_duplicates(subset=['event_id'])

# Filter rows where 'event_id' does NOT start with 'Z7r9jZ1Ad-Vbe'
other_rows = ticket_df1[~ticket_df1['event_id'].str.startswith('Z7r9jZ1Ad')]

# Concatenate the two DataFrames
result_df = pd.concat([distinct_starting_with, other_rows])

# Show the resulting DataFrame
print(result_df)

               event_id                         name  \
46203     Z7r9jZ1Ad_-14  Champlain Valley Exposition   
46204     Z7r9jZ1Ad_-7M       Plymouth Memorial Hall   
46281     Z7r9jZ1Ad_0r6           INTRUST Bank Arena   
46345     Z7r9jZ1Ad_4bO       Riverside Theatre - WI   
46559     Z7r9jZ1Ad_7Fs             White Eagle Hall   
...                 ...                          ...   
46152  3C005EAA8E4E0941        Radio City Music Hall   
46153  3C005EB4CD06154B        Radio City Music Hall   
46154  3C005EBB85A30CB0        Radio City Music Hall   
46161  3C005EC902A510B4        Radio City Music Hall   
46164  3C005EF8CE4410A1        Radio City Music Hall   

                                                   title event_category  \
46203                                        Riley Green          MUSIC   
46204                                          Amy Grant          MUSIC   
46281                    Thomas Rhett: Home Team Tour 23          MUSIC   
46345                      

In [ ]:
# Filter rows where 'event_id' starts with 'Z7r9jZ1Ad-Vbe'
filtered_df = result_df[result_df['event_id'].str.startswith('Z7r9jZ1Ad-_fp')]

# Show the filtered DataFrame
print(filtered_df)

            event_id                           name        title  \
47424  Z7r9jZ1Ad-_fp  Tower Theatre - Oklahoma City  Sleep Token   

      event_category event_subcategory  \
47424          MUSIC             OTHER   

                                 formatted_address  day  month  year  \
47424  425 NW 23rd St, Oklahoma City, OK 73103, US   20      9  2023   

           time                                         event_url       artist  
47424  00:00:00  https://www.ticketmaster.com/event/Z7r9jZ1Ad-_fp  Sleep Token  


# Entity Resolution

main goal is to make sure that the city's names match up

In [ ]:
airbnb_cities = airbnb_inter['city'].unique()
ticket_cities = ticket_inter['city'].unique()
cities = set(airbnb_cities.tolist() + ticket_cities.tolist())
cities
# discrepancies: Atlantic City, Austin, Baltimore, Brooklyn, Chandler, Cincinatti vs. Cincinnati,
# Clinton, Coachella (?), Greensboro, Hot Springs, Las Vegas, Milwaukee, National Harbor, Rapid City,
# Santa Clara, Santa Cruz, Scottsdale, St vs. St., D.C. vs. DC, Winston Salem
# NOTE - i'll change st to st., but d.c. to dc

NameError: name 'airbnb_inter' is not defined

In [ ]:
print("airbnb cities:")
print(airbnb_cities)
print("")
print("ticketmaster cities:")
print(ticket_cities)

airbnb cities:
['San Francisco' 'Washington D.C.' 'Oakland' 'Jersey City' 'New Orleans'
 'Los Angeles' 'New York City' 'Cambridge' 'Santa Clara County'
 'Asheville' 'Salem' 'Columbus' 'Rhode Island' 'San Diego' 'Nashville'
 'Santa Cruz County' 'Denver' 'Chicago' 'Austin' 'Pacific Grove'
 'Portland' 'Seattle' 'Twin Cities MSA' 'Broward County' 'Clark County'
 'Boston' 'San Mateo County']

ticketmaster cities:
['Las Vegas' 'Tucson' 'Des Moines' 'Long Beach' 'Los Angeles'
 'Minneapolis' 'San Diego' 'New Kent' 'Felton' 'Denver' 'Columbia'
 'Columbus' 'Charleston' 'Aurora' 'New York' 'Ontario' 'Englewood'
 'Newport' 'Dayton' 'Cleveland' 'Millvale' 'Salisbury' 'Fort Worth'
 'Asheville' 'National Harbor ' 'Chicago' 'Kinston' 'Portland' 'Logan'
 'Erie' 'Leesburg' 'Evansville' 'Everett' 'Clarkston' 'Boston'
 'Fayetteville' 'Fort Wayne' 'Fredericksburg' 'Fresno' 'Frisco'
 'Portsmouth' 'Ann Arbor' 'Gary' 'Gastonia' 'Orlando' 'Hollywood' 'Omaha'
 'Hampton Beach' 'Dallas' 'South Burlington' 'Greens

In [ ]:
airbnb_inter['city'].replace({
    'Santa Clara County': 'Santa Clara',
    'Santa Cruz County': 'Santa Cruz',
    'San Mateo County': 'San Mateo',
    'Washington D.C.': 'Washington DC'
}, inplace=True)

airbnb_cities = airbnb_inter['city'].unique()
airbnb_cities.tolist()

['San Francisco',
 'Washington DC',
 'Oakland',
 'Jersey City',
 'New Orleans',
 'Los Angeles',
 'New York City',
 'Cambridge',
 'Santa Clara',
 'Asheville',
 'Salem',
 'Columbus',
 'Rhode Island',
 'San Diego',
 'Nashville',
 'Santa Cruz',
 'Denver',
 'Chicago',
 'Austin',
 'Pacific Grove',
 'Portland',
 'Seattle',
 'Twin Cities MSA',
 'Broward County',
 'Clark County',
 'Boston',
 'San Mateo']

In [ ]:
ticket_inter['city'] = ticket_inter['city'].str.replace('St ', 'St. ').str.strip()
ticket_inter['city'] = ticket_inter['city'].str.replace('Ft ', 'Fort ').str.strip()
ticket_inter['city'].replace('St. Paul', 'Twin Cities MSA') # should we put this????
ticket_inter['city'].replace('Winston-Salem', 'Winston Salem', inplace=True)
ticket_cities = ticket_inter['city'].unique()
#ticket_cities.tolist()
set(ticket_cities)

{'Abilene',
 'Adrian',
 'Agoura Hills',
 'Airway Heights',
 'Akron',
 'Albany',
 'Albertville',
 'Albuquerque',
 'Alexandria',
 'Allegan',
 'Allen',
 'Allentown',
 'Allston',
 'Alto',
 'Alton',
 'Altoona',
 'Amarillo',
 'Ames',
 'Amherst',
 'Anaheim',
 'Anchorage',
 'Anderson',
 'Ann Arbor',
 'Annapolis',
 'Apopka',
 'Appleton',
 'Ardmore',
 'Arlington',
 'Arlington Heights',
 'Arrow Rock',
 'Arvada',
 'Asbury Park',
 'Asheville',
 'Ashland',
 'Ashwaubenon',
 'Aspen',
 'Athens',
 'Atlanta',
 'Atlantic City',
 'Atmore',
 'Auburn',
 'Auburn Hills',
 'Augusta',
 'Aurora',
 'Austin',
 'Aventura',
 'Avila Beach',
 'Avon',
 'Avondale',
 'Babylon',
 'Bakersfield',
 'Baltimore',
 'Bangor',
 'Bar Harbor',
 'Baraboo',
 'Baton Rouge',
 'Battle Creek',
 'Bay St. Louis',
 'Bayfield',
 'Beaumont',
 'Beech Mountain',
 'Bellingham',
 'Bellport',
 'Bellvue',
 'Belmont Park - Long Island',
 'Bemidji',
 'Bend',
 'Bensalem',
 'Berea',
 'Berkeley',
 'Bethel',
 'Bethlehem',
 'Beverly',
 'Beverly Hills',
 'B

In [ ]:
cities = set(airbnb_cities.tolist() + ticket_cities.tolist())
cities # now the cities look okay

{'Abilene',
 'Adrian',
 'Agoura Hills',
 'Airway Heights',
 'Akron',
 'Albany',
 'Albertville',
 'Albuquerque',
 'Alexandria',
 'Allegan',
 'Allen',
 'Allentown',
 'Allston',
 'Alto',
 'Alton',
 'Altoona',
 'Amarillo',
 'Ames',
 'Amherst',
 'Anaheim',
 'Anchorage',
 'Anderson',
 'Ann Arbor',
 'Annapolis',
 'Apopka',
 'Appleton',
 'Ardmore',
 'Arlington',
 'Arlington Heights',
 'Arrow Rock',
 'Arvada',
 'Asbury Park',
 'Asheville',
 'Ashland',
 'Ashwaubenon',
 'Aspen',
 'Athens',
 'Atlanta',
 'Atlantic City',
 'Atmore',
 'Auburn',
 'Auburn Hills',
 'Augusta',
 'Aurora',
 'Austin',
 'Aventura',
 'Avila Beach',
 'Avon',
 'Avondale',
 'Babylon',
 'Bakersfield',
 'Baltimore',
 'Bangor',
 'Bar Harbor',
 'Baraboo',
 'Baton Rouge',
 'Battle Creek',
 'Bay St. Louis',
 'Bayfield',
 'Beaumont',
 'Beech Mountain',
 'Bellingham',
 'Bellport',
 'Bellvue',
 'Belmont Park - Long Island',
 'Bemidji',
 'Bend',
 'Bensalem',
 'Berea',
 'Berkeley',
 'Bethel',
 'Bethlehem',
 'Beverly',
 'Beverly Hills',
 'B

# Finding Indices

mainly for spotify, since there's no specified id

In [ ]:
# determining which columns - rank, date, region
grouped_spotify = spotify_inter.groupby(["rank", "date", "region"])
counts = grouped_spotify.size()
print(counts)
#(counts == 1).all()

rank  date        region       
1     2017-01-01  United States    1
      2017-01-02  United States    1
      2017-01-03  United States    1
      2017-01-04  United States    1
      2017-01-05  United States    1
                                  ..
200   2021-12-27  United States    1
      2021-12-28  United States    1
      2021-12-29  United States    1
      2021-12-30  United States    1
      2021-12-31  United States    2
Length: 364184, dtype: int64




---

# Exporting

In [ ]:
airbnb_inter.to_csv("airbnb_cleaned.csv", index=False)
spotify_inter.to_csv("spotify_cleaned.csv", index=False)
ticket_inter.to_csv("ticketmaster_cleaned.csv", index=False)

In [ ]:
from google.colab import files
files.download("airbnb_cleaned.csv")
files.download("spotify_cleaned.csv")
files.download("ticketmaster_cleaned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [57]:
from google.colab import files
airbnb_df1.to_csv("airbnb_main_cleaned.csv", index=False)
airbnb_df2.to_csv("airbnb_host_cleaned.csv", index=False)
airbnb_df3.to_csv("airbnb_latlong_cleaned.csv", index=False)
files.download("airbnb_main_cleaned.csv")
files.download("airbnb_host_cleaned.csv")
files.download("airbnb_latlong_cleaned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [63]:
from google.colab import files
spotify_df1.to_csv("spotify_main_cleaned.csv", index=False)
spotify_df2.to_csv("spotify_url_cleaned.csv", index=False)
files.download("spotify_main_cleaned.csv")
files.download("spotify_url_cleaned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download("airbnb_main_cleaned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
ticket_df1.to_csv("ticket_main_cleaned.csv", index=False)
ticket_df2.to_csv("ticket_addr_cleaned.csv", index=False)
files.download("ticket_main_cleaned.csv")
files.download("ticket_addr_cleaned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
spotify_df = pd.read_csv('/content/drive/MyDrive/NewChart.csv')

In [ ]:
spotify_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171327 entries, 0 to 171326
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   title       171327 non-null  object
 1   chart_rank  171327 non-null  int64 
 2   date        171327 non-null  object
 3   artist      171327 non-null  object
 4   url         171327 non-null  object
 5   trend       171327 non-null  object
 6   streams     171327 non-null  int64 
 7   day         171327 non-null  int64 
 8   month       171327 non-null  int64 
 9   year        171327 non-null  int64 
dtypes: int64(5), object(5)
memory usage: 13.1+ MB


In [ ]:
spotify_df.drop('date', axis=1)

,title,chart_rank,artist,url,trend,streams,day,month,year
0,!,173,Trippie Redd,https://open.spotify.com/track/1qtHdSxiavDgUPZ...,MOVE_DOWN,220775,15,8,2019
1,!,13,Trippie Redd,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,NEW_ENTRY,808145,9,8,2019
2,!,42,Trippie Redd,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,MOVE_DOWN,442794,10,8,2019
3,!,61,Trippie Redd,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,MOVE_DOWN,331191,11,8,2019
4,!,77,Trippie Redd,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,MOVE_DOWN,315318,12,8,2019
...,...,...,...,...,...,...,...,...,...
171322,ZTFO,99,Big Sean,https://open.spotify.com/track/0z36JaXXoFXuPuQ...,MOVE_DOWN,283505,7,9,2020
171323,ZTFO,130,Big Sean,https://open.spotify.com/track/0z36JaXXoFXuPuQ...,MOVE_DOWN,246576,8,9,2020
171324,ZTFO,154,Big Sean,https://open.spotify.com/track/0z36JaXXoFXuPuQ...,MOVE_DOWN,230297,9,9,2020
171325,ZTFO,170,Big Sean,https://open.spotify.com/track/0z36JaXXoFXuPuQ...,MOVE_DOWN,216578,10,9,2020


In [ ]:
url_title_df = spotify_df[['url', 'title', 'artist']]
url_title_df

,url,title,artist
0,https://open.spotify.com/track/1qtHdSxiavDgUPZ...,!,Trippie Redd
1,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,!,Trippie Redd
2,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,!,Trippie Redd
3,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,!,Trippie Redd
4,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,!,Trippie Redd
...,...,...,...
171322,https://open.spotify.com/track/0z36JaXXoFXuPuQ...,ZTFO,Big Sean
171323,https://open.spotify.com/track/0z36JaXXoFXuPuQ...,ZTFO,Big Sean
171324,https://open.spotify.com/track/0z36JaXXoFXuPuQ...,ZTFO,Big Sean
171325,https://open.spotify.com/track/0z36JaXXoFXuPuQ...,ZTFO,Big Sean


In [ ]:
url_title_df['url'].nunique()

4132

In [ ]:
url_title_df['title'].nunique()

3521

In [ ]:
url_title_df['artist'].nunique()

933

In [ ]:
url_title_df = url_title_df.drop_duplicates()
url_title_df

,url,title,artist
0,https://open.spotify.com/track/1qtHdSxiavDgUPZ...,!,Trippie Redd
1,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,!,Trippie Redd
7,https://open.spotify.com/track/6hklGK8K5sqP4IK...,¿Donde Has Estado?,Eslabon Armado
8,https://open.spotify.com/track/41LhQUkElADQ5YU...,"...And To Those I Love, Thanks For Sticking Ar...",$uicideboy$
142,https://open.spotify.com/track/4GBkffrtA51p17J...,‘tis the damn season,Taylor Swift
...,...,...,...
171301,https://open.spotify.com/track/6vn5wV2qwx5seng...,Zombie (feat. NLE Choppa & DB Omerta),DB Omerta
171305,https://open.spotify.com/track/6vn5wV2qwx5seng...,Zombie (feat. NLE Choppa & DB Omerta),Kodak Black
171309,https://open.spotify.com/track/3Ddny34bFoYBb8d...,Zoo York (feat. Fivio Foreign & Pop Smoke),Lil Tjay
171319,https://open.spotify.com/track/0z36JaXXoFXuPuQ...,ZTFO,Big Sean


In [ ]:
from google.colab import files
url_title_df.to_csv("ChartUrl.csv", index=False)
files.download("ChartUrl.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
spotify_df = spotify_df.drop('date', axis=1)

In [ ]:
spotify_df

,chart_rank,artist,url,trend,streams,day,month,year
0,173,Trippie Redd,https://open.spotify.com/track/1qtHdSxiavDgUPZ...,MOVE_DOWN,220775,15,8,2019
1,13,Trippie Redd,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,NEW_ENTRY,808145,9,8,2019
2,42,Trippie Redd,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,MOVE_DOWN,442794,10,8,2019
3,61,Trippie Redd,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,MOVE_DOWN,331191,11,8,2019
4,77,Trippie Redd,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,MOVE_DOWN,315318,12,8,2019
...,...,...,...,...,...,...,...,...
171322,99,Big Sean,https://open.spotify.com/track/0z36JaXXoFXuPuQ...,MOVE_DOWN,283505,7,9,2020
171323,130,Big Sean,https://open.spotify.com/track/0z36JaXXoFXuPuQ...,MOVE_DOWN,246576,8,9,2020
171324,154,Big Sean,https://open.spotify.com/track/0z36JaXXoFXuPuQ...,MOVE_DOWN,230297,9,9,2020
171325,170,Big Sean,https://open.spotify.com/track/0z36JaXXoFXuPuQ...,MOVE_DOWN,216578,10,9,2020


In [ ]:
spotify_df = spotify_df.drop_duplicates()

In [ ]:
spotify_df = spotify_df.drop('artist', axis=1)

In [ ]:
spotify_df.to_csv("ChartMain.csv", index=False)
files.download("ChartMain.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>